In [13]:
import json
from pathlib import Path

import pandas as pd

- Import the CSV, then sanitize the column names.
- Adjust `csv_inpath` as appropriate.
- Rename columns for clarity.

In [14]:
# Load the CSV-formatted use cases from a file.
csv_inpath = Path("data/input/20250516-autodesk-jira-tickets-updated-sheet.csv")
df = pd.read_csv(csv_inpath, skiprows=1, usecols=[0, 3, 4, 6, 7, 8])
renames = {
    "Use Case Title": "use_case_title",  # This cust. data has an extra space.
    "ticket number": "ticket_number",
    "jira ticket": "jira_ticket",
    "Prompts": "prompts",
    "Purpose of Use": "purpose_of_use",
    "Response to Request": "response_to_request",
}
df.rename(columns=renames, inplace=True)
df.head()  # Uncomment to check input. Be sure not to share sensitive data.

,use_case_title,ticket_number,jira_ticket,prompts,purpose_of_use,response_to_request
0,Image resolution for Visual Production Artists,87,https://jira.autodesk.com/browse/GENAIGOV-87,customer-facing features' thumbnailsexamples: ...,While creating a feature pages for our Autodes...,Please work with your LBP to ensure that the a...
1,Creating Content Clusters for Industry Specifi...,144,https://jira.autodesk.com/browse/GENAIGOV-144,persona based prompts: Generate an Outreach em...,create content clusters - industry specific co...,Your request to use Abacus Agent AI and ChatLL...
2,Developing learning materials,201,https://jira.autodesk.com/browse/GENAIGOV-201,"text descriptions, documentation",developing learning materials,Your request to use the Absorb AI content gene...
3,"image and text generation in photoshop, illust...",157,https://jira.autodesk.com/browse/GENAIGOV-157,Generative AI image and text creations in Adob...,"image and text generation in photoshop, illust...","The described use case is approved; provided, ..."
4,increase image resolution (tool uses GAN),155,https://jira.autodesk.com/browse/GENAIGOV-155,Software screenshots or video stills that are ...,increase image resolution (tool uses GAN),"The described use case is approved; provided, ..."


Iterate down the rows of the CSV, transforming each one into a JSON object defined by [`use-case-schema.json`](./use-case-schema.json).

In [15]:
# status_mapping = {"Approved": 1, "Pending": 0, "Rejected": -1}
use_cases = []
# now_iso = datetime.now(timezone.utc).isoformat()

# Row by row, transform each CSV-formatted use case into a JSON-formatted dict.
for _, row in df.iterrows():
    use_case = {
        # "id": str(uuid.uuid4()),
        "name": row["use_case_title"].strip(),
        "description": (
            f"Purpose of Use:\n{row['purpose_of_use']}\n\n"
            f"Response to Request:\n{row['response_to_request']}\n\n"
            f"Prompts:\n{row['prompts']}\n\n"
            f"Ticket Number: {row['ticket_number']}\n"
            f"JIRA Ticket: {row['jira_ticket']}"
        ),
        # "icon": None,
        # "ai_type": "",
        # "governance_status": status_mapping.get(row["status"].strip(), 0),
        # "domains": [],
        # "industries": [],
        # "regions": [],
        # "risk_category_level": None,
        # "custom_fields": [],
        # "questionnaires": [],
        # "inserted_at": now_iso,
        # "updated_at": now_iso,
    }
    use_cases.append(use_case)

# Save the list of JSON-formatted (dict) use cases to a file.
outfolder = csv_inpath.parents[1] / "output"
outfolder.mkdir(parents=True, exist_ok=True)
filename = csv_inpath.with_name(csv_inpath.stem + "-bulk-upload.json").name
json_outpath = outfolder / filename
payload = {"data": {"items": use_cases}}
with open(json_outpath, "w") as f:
    json.dump(payload, f, indent=4)